In [1]:
import pandas as pd
import numpy as np

from neurotools.stats.permutations import permuted_v
from neurotools.loading.abcd import load_from_csv, load_family_block_structure

- Load data

For this example problem, we will just use a set of thickness ROI's, just family id as the block structure, and a single confounding variable of sex, and let's say neurocog1 as our target variable.

In [2]:
# Get possible columns
csv_loc = '/home/sage/benchmark_methods/data/nda3.0.csv'
all_cols = list(pd.read_csv(csv_loc, nrows=0))

# Load thickness ROI data - ignore hemi and full mean for plotting
thick = [col for col in all_cols if 'smri_thick_cort.destrieux' in col and '_mean' not in col]

# Rest of needed columns
rest = ['C(sex_at_birth)', 'neurocog_pc1.bl', 'C(rel_family_id)']

# Load in all together
data = load_from_csv(cols=thick+rest, csv_loc=csv_loc, drop_nan=True)

Note the parameter within_grp. If set to True, then means swaps will only be performed within group, so in terms of family, that means the only swaps that would take place are within families, swapping siblings data with each other. If set to False, which we do in this example, it instead means that swaps occur between groups, so every singleton family is freely swapped with other singleton families, but any family of size 2, can only be swapped with another family of size 2.

See: https://sahahn.github.io/neurotools/stats.html#permutations-permuted-v
For more information on this method.

In [ ]:
pvals, original_scores, h0_vmax = permuted_v(tested_vars=data['neurocog_pc1.bl'],
                                             target_vars=data[thick],
                                             confounding_vars=data['sex_at_birth'],
                                             permutation_structure=data[['rel_family_id']],
                                             within_grp=False,
                                             n_perm=100,
                                             verbose=1,
                                             n_jobs=4,
                                            )

[ 0  1  2  3  4  5  6  7  8  9 10]


In [7]:
np.unique(np.unique(data['rel_family_id'], return_counts=True)[1])

array([1, 2, 3, 5])

In [ ]:
from neurotools.plotting import SurfRef, plot

def to_vertex(vals, names, space='fsaverage5'):
    '''Helper function to convert destr. ROIs to vertex space'''
    
    plot_df = pd.DataFrame(vals, columns=['vals'])
    plot_df['names'] = names

    surf_ref = SurfRef(space=space, parc='destr')
    plot_data = surf_ref.get_hemis_plot_vals(plot_df, lh_key='.lh', rh_key='.rh')
    
    return plot_data

In [ ]:
roi_names = list(data[thick])
v = to_vertex(original_scores, roi_names)
plot(v, title='Raw v-stats', threshold=.1)

In [ ]:
original_scores

In [ ]:
h0_vmax